In [ ]:
from torch.optim import Adam

from torchkge.evaluation import LinkPredictionEvaluator
from torchkge.models import TransEModel
from torchkge.utils.datasets import load_fb15k
from torchkge.utils import Trainer, MarginLoss

In [ ]:
import pandas as pd
import numpy as np
import os

from torchkge.models import ConvKBModel
from torchkge.utils.datasets import load_fb15k
import pickle 
from copy import deepcopy
from tqdm import tqdm 

from torch import cuda
from torch.optim import Adam

from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader

from tqdm.autonotebook import tqdm

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [ ]:
# Load dataset
kg_train, _, _ = load_fb15k()

In [ ]:
class Config:
    kge_n_epoch = 100
    kge_lr = 1e-4
    kge_batch = 64
    kge_margin = 0.5
    kge_conv_size = 3

In [ ]:
config = Config()
margin = 0.5
n_epochs = 1000
emb_dim = 100
lr = 0.0004
b_size = 32768
conv_size = 64

In [ ]:
model = ConvKBModel(emb_dim, conv_size, kg_train.n_ent, kg_train.n_rel)
criterion = MarginLoss(margin)

# Move everything to CUDA if available
if cuda.is_available():
    cuda.empty_cache()
    model.cuda()
    criterion.cuda()

# Define the torch optimizer to be used
optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)

sampler = BernoulliNegativeSampler(kg_train)
dataloader = DataLoader(kg_train, batch_size=b_size, use_cuda='all')

iterator = tqdm(range(n_epochs), unit='epoch')
for epoch in iterator:
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        h, t, r = batch[0], batch[1], batch[2]
        n_h, n_t = sampler.corrupt_batch(h, t, r)

        optimizer.zero_grad()

        # forward + backward + optimize
        pos, neg = model(h, t, r, n_h, n_t)
        loss = criterion(pos, neg)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    iterator.set_description(
        'Epoch {} | mean loss: {:.5f}'.format(epoch + 1,
                                              running_loss / len(dataloader)))

model.normalize_parameters()

In [ ]:
model_convKB.scoring_function(3,7,7)

In [ ]:
import torchkge
import torch
e1 = torch.LongTensor([10, 19])
e2 = torch.LongTensor([997, 999])
result = torchkge.inference.RelationInference(model_convKB, e1, e2, top_k=3)

In [ ]:
result.predictions

In [ ]:
result.scores

In [ ]:
kg.head_idx

In [ ]:
kg.ent2ix